Created on Aug 8 2024

@author: Francesco Maria Bellussi

In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import fft
import matplotlib.pyplot as plt

# VACF Spectral Overlap

In [ ]:
Fs = 1/(1*10**-15)          # Sampling frequency
T = 1/Fs                    # Sampling period
L = 10001                   # Length of signal
t = np.arange(0, L) * T     # Time vector

# Initialize a list to store each P1 spectrum
P1_list = []
P12_list = []

for i in range(1, 101):
    # Read data from file
    filename_water = f'../../lammps/PDOS/vacf_water.{i}'
    T = pd.read_table(filename_water, delimiter=' ', comment='#', header=None)
    V = T.iloc[:, 1].values
    
    filename_graph = f'../../lammps/PDOS/vacf_graph.{i}'
    T2 = pd.read_table(filename_graph, delimiter=' ', comment='#', header=None)
    V2 = T2.iloc[:, 1].values
    
    # Perform FFT
    Y = fft(V)
    Y2 = fft(V2)
    
    P2 = np.abs(Y/L)
    P1 = P2[:L//2+1]
    P1[1:-1] = 2 * P1[1:-1]
    
    P22 = np.abs(Y2/L)
    P12 = P22[:L//2+1]
    P12[1:-1] = 2 * P12[1:-1]
    
    # Store P1 in the list
    P1_list.append(P1)
    P12_list.append(P12)
    
    # # Save individual results to file
    f = Fs * np.arange(0, (L//2)+1) / (L * 10**12)  # Frequency vector
    # A = np.vstack((f, P1)).T
    # file_name = f'file_g{i}.txt'
    # np.savetxt(file_name, A, fmt='%d %d', header='Frequency(THz) Amplitude')

    # # Plot and save individual figure
    # plt.figure()
    # plt.plot(f, P1, '-')
    # plt.title(f'Single-Sided Amplitude Spectrum of X(t) - {i}')
    # plt.xlabel('f (THz)')
    # plt.ylabel('|DOF|')
    # plt.savefig(f'vacf_pla_{i}.png')  # Save as PNG file
    # plt.close()

# Compute average P1 across all spectra
P1_avg = np.mean(P1_list, axis=0)
P12_avg = np.mean(P12_list, axis=0)

df=f[1]-f[0]

S_graph_water=(np.sum(P1_avg*P12_avg)*df)/((np.sum(P1_avg)*np.sum(P12_avg))*df**2)

print("Overlap Graphene/water : {} ".format(S_graph_water))

# Plot

In [ ]:
cm = 1/2.54
plt.figure(figsize=(18*cm, 15*cm))
plt.plot(f, P1_avg, '-',color='aqua',label='Water')
plt.plot(f, P12_avg, '-',color='maroon',label='Graphene')
plt.fill_between(f,P1_avg,color='aqua',alpha=0.3)
plt.fill_between(f,P12_avg,color='maroon',alpha=0.3)
plt.title('Graphene', x=0.5, y=0.9,fontsize=50*cm)
plt.title('Average Single-Sided Amplitude Spectrum of X(t)')
plt.xlabel('frequency (THz)',fontsize=50*cm)
plt.ylabel('Phonon DOS (a.u.)',fontsize=50*cm)
plt.tick_params(axis='x', labelsize=50*cm)
plt.tick_params(axis='y', labelsize=50*cm)
plt.tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False)
plt.xlim([0, 200])  # Limit the x-axis to 200 THz

plt.legend(fontsize=50*cm,loc='upper right')

plt.tight_layout()

plt.savefig('dos_water-graph.png')  # Save as PNG file
plt.savefig('dos_water-graph.pdf')  # Save as PDF file

plt.show()